# Substitutionschiffre

In [ ]:
import numpy as np
from functools import lru_cache

from collections import defaultdict
from itertools import permutations
from urllib.request import urlopen

In [ ]:
with open("p1.txt", "r") as fd:
    raw = fd.read()
raw

In [ ]:
@lru_cache(maxsize=10)
def fetchBook(url):
    with urlopen(url) as fd:
        text = fd.read().decode("utf-8").replace("\xa0", " ").lower()
        return " ".join(text.split("\r\n"))[1:] # Remove BOM
    
ENG = fetchBook("https://www.gutenberg.org/cache/epub/394/pg394.txt").lower()
#DEU = fetchBook("https://www.gutenberg.org/cache/epub/6343/pg6343.txt").lower()

In [ ]:
def ngrams(s, n):
    for line in s.split("\n"):
        for ngram in [line[i:(i+n)] for i in range(len(line) - n)]:
            yield ngram

In [ ]:
logLik = defaultdict(int)
total = 0
for ngram in ngrams(ENG, 4):
    logLik[ngram] += 1
    total += 1
    
for ngram in logLik.keys():
    logLik[ngram] = np.log(logLik[ngram] / total)

In [ ]:
def likelihood(text):
    ll = 0.0
    for ngram in ngrams(text, 4):
        ll += logLik[ngram]
    return ll

In [ ]:
raw[:20]

In [ ]:
msg_alphabet

In [ ]:
abc = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
msg_alphabet = np.unique(list(ngrams(raw, 1)) + abc)
msg_alphabet

In [ ]:
xlat = dict(zip(msg_alphabet, msg_alphabet))
xlat["'"], xlat['s'] = xlat['s'], xlat["'"]
xlat["k"], xlat[','] = xlat[','], xlat["k"]

bestLL = likelihood(raw.translate(str.maketrans(xlat)))
bestXlat = xlat.copy()

In [ ]:
%%time

for i in range(5):
    xlat = dict(zip(msg_alphabet, msg_alphabet))
    xlat["'"], xlat['s'] = xlat['s'], xlat["'"]
    xlat["k"], xlat[','] = xlat[','], xlat["k"]
    stall = False
    while not stall:
        stall = True
        for p in np.random.permutation(msg_alphabet):
            for q in np.random.permutation(msg_alphabet):
                if p == q:
                    next
                xlat[p], xlat[q] = xlat[q], xlat[p]    
                ll = likelihood(raw.translate(str.maketrans(xlat)))
                if ll < bestLL:            
                    bestLL, bestXlat = ll, xlat.copy()
                    stall = False
                elif ll > bestLL:                    
                    xlat[p], xlat[q] = xlat[q], xlat[p]
    print(f"Attempt {i}: {bestLL}")

In [ ]:
plain = raw.translate(str.maketrans(bestXlat))
print(plain)

In [ ]:
N = 151
print(f"https://en.wikipedia.org/wiki/index.php?curid={N * 8}")

In [ ]:
z = np.array(list(ngrams(plain, 1)))
for p in permutations(["0", "1", "6", "8"]):
    id = int("".join(p)) + np.sum(z == 'e') - np.sum(z == 't') + np.sum(z == 'a') - np.sum(z == 'o') + np.sum(z == 'i') - np.sum(z == 'n')


In [ ]:
np.sum(z == 'e')

In [ ]:
np.sum(z == 't')

In [ ]:
np.sum(z == 'a')

In [ ]:
np.sum(z == 'o')

In [ ]:
np.sum(z == 'i')

In [ ]:
np.sum(z == 'n')

In [ ]:
1086 + 443 - 330 + 265 - 258 + 220 - 215